# HowTo -Voxels- From mask images and camera coordinates to volume

In this notebook you will learn:

1. how to connect to a ROMI database
2. how to use our custom space-carving method to reconstruct the volume
3. how to visualize the reconstructed volume

This notebook **assume** that you have:
- declared the `ROMI_DB` environment variable as the path to the database directory to use
- processed the test dataset with the _geometric pipeline,_ so we can access the fileset containing the data we want to start with...

Remember, the aim of this notebook is to show you how it works "under the hood".
This is not how you should process your data, that is done thanks to the `romi_run_task` CLI tool.

In [ ]:
import os

import ipywidgets as widgets
from plant3dvision.cl import Backprojection
from plant3dvision.visu import plotly_volume_slicer, plotly_image_carousel
from plantdb import FSDB

## Connect to the database & get the initial data

If you did not declare a `ROMI_DB` environment variable, you can do it by uncommenting the next cell and setting it to the right value.

In [ ]:
# os.environ['ROMI_DB'] = "/path/to/test/data"

### Connect to the database

In [ ]:
db = FSDB(os.environ['ROMI_DB'])  # requires definition of this environment variable!
db.connect()

Once you are connected to the database, you can list the available scan *dataset* with `db.list_scans()`.

### Select a dataset

We now select a dataset (with the `Dropdown` widget) for the demo:

In [ ]:
scan_name = widgets.Dropdown(options=db.list_scans(), value=db.list_scans()[0], description='Dataset:')
display(scan_name)

In [ ]:
scan = db.get_scan(scan_name.value)

If you did not process this dataset yet, from the `plant3dvision` root directory, you can do it with:
```
romi_run_task AnglesAndInternodes $ROMI_DB/<selected_dataset> --config plant-3d-vision/configs/geom_pipe_real.toml
```

To list the available *filesets* in this *scan dataset*:

In [ ]:
scan.list_filesets()

### Get the binary mask images fileset

The binary mask images resulting from a _Masks_ task are to be found in the 'Masks*' fileset.

In [ ]:
mask_fs = scan.get_filesets(query={"task_name": "Masks"})[0]
print(mask_fs.path().stem)

Once you have access to the 'Masks*' fileset, you may access the mask images as follows:

In [ ]:
masks_files = mask_fs.get_files()

In [ ]:
print(f"This fileset contains {len(masks_files)} files (matching the `query`).")

### Visualize the set of mask images

It is possible to visualize the set of RGB images using our `plotly_image_carousel` method.

In [ ]:
fig = plotly_image_carousel(masks_files, title=scan_name.value)

In [ ]:
fig.show()

## Reconstruct the volume

### Define the voxel carving parameters

We start by defining what part of the scene, as estimated by Colmap, we would like to reconstruct as a volume.
Obviously this volume should contain the plant.

Finding those parameters' value can be done by using the reconstructed sparse point cloud.
A good acquisition setup will help in preventing the need to re-estimate these for every acquisition.

In [ ]:
bounding_box = {
    "x": [300, 450],
    "y": [300, 450],
    "z": [-175, 100],
}
voxel_size = 0.5

### Define the shape & origin of the voxel array

In [ ]:
x_min, x_max = bounding_box["x"]
y_min, y_max = bounding_box["y"]
z_min, z_max = bounding_box["z"]

nx = int((x_max - x_min) / voxel_size) + 1
ny = int((y_max - y_min) / voxel_size) + 1
nz = int((z_max - z_min) / voxel_size) + 1

### Perform volume carving

You may notice we **do not** have to provide the camera poses estimated by the `Colmap` task.
These estimated poses are saved as metadata and are accessed directly by `plant3dvision.cl.Backprojection`.

In [ ]:
space_carving = Backprojection(shape=[nx, ny, nz], origin=[x_min, y_min, z_min], voxel_size=voxel_size)

In [ ]:
vol = space_carving.process_fileset(masks_files, camera_metadata='colmap_camera')

### Visualize the carved volume

Once the volume carving is done, you may visualize it with the `plotly_volume_slicer` from `plant3dvision.visu` as follows:

In [ ]:
fig = plotly_volume_slicer(vol)
fig.show()

We may now **disconnect** from the database as we will not need it anymore:

In [ ]:
db.disconnect()